In [11]:
import pandas as pd

# users = pd.read_csv('data/users_export.csv')
ratings = pd.read_csv('data/ratings_export.csv')
movies = pd.read_csv('data/movie_data.csv', on_bad_lines='skip')
my_ratings = pd.read_csv('data/processed/ratings_tmdb_cleaned.csv')

# Merge datasets to create a comprehensive dataset
user_ratings = pd.merge(ratings, movies, left_on='tmdb_id', right_on='movie_id')

In [12]:
user_ratings.head()

,_id_x,movie_id,rating_val,user_id,_id_y,genres,image_url,imdb_id,imdb_link,movie_title,...,popularity,production_countries,release_date,runtime,spoken_languages,tmdb_id,tmdb_link,vote_average,vote_count,year_released
0,5fc57c5d6758f6963451a07f,feast-2014,7,deathproof,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
1,5fc57ca06758f69634538bad,feast-2014,10,ingridgoeswest,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
2,5fc57cbd6758f696345475a3,feast-2014,8,dirkh,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
3,5fc57ce06758f6963455400b,feast-2014,10,childrenofmen,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
4,5fc57cf36758f69634558b0e,feast-2014,8,suspirliam,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0


In [13]:
len(user_ratings)

11079666

In [14]:
len(my_ratings)

186

In [24]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

# Assuming 'tmdb_id' in movies DataFrame and 'id' in ratings DataFrame are the columns to be used
# The 'left_on' and 'right_on' parameters in merge function need to be updated accordingly

# Updated merge to use 'tmdb_id' from movies DataFrame
merged = my_ratings.merge(movies[['tmdb_id', 'movie_title', 'year_released']], 
                          left_on='id',  # Update this based on the actual column in my_ratings that matches tmdb_id in movies
                          right_on='tmdb_id', 
                          how='inner')

# remove ratings of 0 (not rated)
merged = merged[merged['Rating'] != 0]

my_ratings_updated = merged[['tmdb_id', 'Rating']].copy()  # Use .copy() to explicitly create a copy
my_ratings_updated['user_id'] = 0  # Assign 'user_id' = 0 directly

# Aggregate ratings for each movie (in case of duplicates), using 'tmdb_id'
my_ratings_updated = my_ratings_updated.groupby('tmdb_id').agg({'Rating': 'mean'}).reset_index()

my_ratings_updated, len(my_ratings_updated)


(     tmdb_id  Rating
 0       11.0     4.5
 1       12.0     4.5
 2       85.0     3.5
 3       89.0     3.5
 4      105.0     4.0
 ..       ...     ...
 69  593643.0     1.0
 70  603692.0     3.5
 71  616037.0     3.0
 72  673593.0     2.5
 73  695721.0     3.5
 
 [74 rows x 2 columns],
 74)

In [25]:
# Add your ratings to the user ratings DataFrame
combined_ratings = pd.concat([user_ratings, my_ratings_updated.rename(columns={'Rating': 'rating_val'})])

# Map movie IDs and user IDs to indices for creating a sparse matrix
tmdb_id_to_idx = {tmdb_id: i for i, tmdb_id in enumerate(combined_ratings['tmdb_id'].unique())}
user_id_to_idx = {user_id: i for i, user_id in enumerate(combined_ratings['user_id'].unique())}

# Create a sparse matrix of ratings
rows = combined_ratings['user_id'].map(user_id_to_idx)
cols = combined_ratings['tmdb_id'].map(tmdb_id_to_idx)
data = combined_ratings['rating_val']
ratings_matrix = csr_matrix((data, (rows, cols)), shape=(len(user_id_to_idx), len(tmdb_id_to_idx)))

# Compute cosine similarity between users
user_similarity = cosine_similarity(ratings_matrix)
user_similarity

In [ ]:
# Since you are user_id 0, find the most similar users to you
top_100_similar_users_indices = np.argsort(-user_similarity[0])[1:101]  # Get indices of top 100 similar users

my_rated_movies = set(my_ratings_updated['tmdb_id'])

# Initialize a dictionary to hold recommended movies and the users who recommended them
recommended_movies_details = {}

for user_index in top_100_similar_users_indices:
    user_id = list(user_id_to_idx.keys())[list(user_id_to_idx.values()).index(user_index)]
    
    # # Filter movies rated 5/5 by the user
    # high_rated_movies_by_user = combined_ratings[(combined_ratings['user_id'] == user_id) & (combined_ratings['rating_val'] == 5)]
    high_rated_movies_by_user = combined_ratings[(combined_ratings['user_id'] == user_id)]
    
    for _, row in high_rated_movies_by_user.iterrows():
        tmdb_id = row['tmdb_id']
        if tmdb_id not in my_rated_movies:
            if tmdb_id not in recommended_movies_details:
                recommended_movies_details[tmdb_id] = {'users': [user_id], 'ratings': [row['rating_val']]}
            else:
                recommended_movies_details[tmdb_id]['users'].append(user_id)
                recommended_movies_details[tmdb_id]['ratings'].append(row['rating_val'])

# Limit to top 5 based on the number of users recommending the movie
recommended_movies_ids = sorted(recommended_movies_details, key=lambda x: len(recommended_movies_details[x]['users']), reverse=True)[:100]

# Fetch movie titles and stats
recommended_titles_and_stats = []
for tmdb_id in recommended_movies_ids:
    movie_title = movies[movies['tmdb_id'] == tmdb_id]['movie_title'].iloc[0]
    avg_rating = np.mean(recommended_movies_details[tmdb_id]['ratings'])
    num_users = len(recommended_movies_details[tmdb_id]['users'])
    recommended_titles_and_stats.append({
        'title': movie_title,
        'average_rating': avg_rating,
        'recommended_by_users_count': num_users,
        'recommended_by_user_ids': recommended_movies_details[tmdb_id]['users']
    })

# Display recommended movies along with stats
for movie in recommended_titles_and_stats:
    print(f"Title: {movie['title']}, Avg Rating: {movie['average_rating']:.2f}, Recommended by {movie['recommended_by_users_count']} Users, User IDs: {movie['recommended_by_user_ids']}")

Title: Ex Machina, Avg Rating: 8.66, Recommended by 169 Users, User IDs: ['bartonfinked', 'bartonfinked', 'childrenofmen', 'childrenofmen', 'kwang', 'kwang', 'lukelivermore', 'alicjas', 'alicjas', 'kevschwa', 'kevschwa', 'phntmthrd', 'phntmthrd', 'sushifern', 'sushifern', 'ianamurray', 'joshuatbrown', 'joshuatbrown', 'hotdad23', 'hotdad23', 'mrmichaeldjones', 'mrmichaeldjones', 'hewasthrfriendd', 'hewasthrfriendd', 'ingridgoeswest', 'ingridgoeswest', 'samkohn', 'samkohn', 'lesaladino', 'lesaladino', 'sadiemuriel', 'sadiemuriel', 'jessiebuckley', 'akroeker', 'livmallia', 'livmallia', 'tburnett217', 'zaneg4l', 'zaneg4l', 'haber345', 'haber345', 'zyopy', 'zyopy', 'amandaayoung', 'ezracubero', 'ezracubero', 'vprento', 'vprento', 'sonofjorel', 'sonofjorel', 'dazapata', 'dazapata', 'thecinemalover', 'mouserat', 'mknightshymalan', 'mknightshymalan', 'potato_digest', 'potato_digest', 'tracesauveur', 'tracesauveur', 'babydriverreal', 'bencaplan', 'tommysyk', 'tommysyk', 'hereditary', 'hereditar

In [ ]:
# output to file

with open('data/recommended_movies.txt', 'w') as f:
    for movie in recommended_titles_and_stats:
        f.write(f"Title: {movie['title']}, Avg Rating: {movie['average_rating']:.2f}, Recommended by {movie['recommended_by_users_count']} Users, User IDs: {movie['recommended_by_user_ids']}\n")